# introduccion

#### Como trabajo parcial, para el curso de complejidad algorítmica, se implementará el juego Quoridor en el lenguaje de programación Python. El juego consiste en llegar al extremo opuesto de tablero antes que los demás jugadores, para ello los jugadores podrán poner barreras con el objetivo de bloquear los caminos de los demás jugadores y así poder llegar antes al extremo opuesto. Además, los jugadores solo pueden moverse un solo espacio (arriba, abajo, derecha o izquierda), saltar encima de una ficha, si la ficha del otro jugador esta al costado), moverse en diagonal si al querer saltar encima de una ficha existe una barrera en el siguiente espacio o poner una barrera. Para el desarrollo de este proyecto se usará Jupyter Notebook el cual se usará para explicar detalladamente el código a realizar. También se usará GitHub para poder tener un correcto control de versiones y poder trabajar de manera colaborativa. Por último, se usará la plataforma Discord para comunicarnos como equipo.